In [95]:
# Define paths to model files
import os
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

In [96]:
import tensorflow as tf
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [97]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [98]:
# define input sequence
Sensor1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
Sensor2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
Sensor3 = array([20, 30, 40, 50, 60, 70, 80, 90, 100])
out_seq = array([Sensor1[i]+Sensor2[i]-Sensor3[i] for i in range(len(Sensor1))])
# convert to [rows, columns] structure
Sensor1 = Sensor1.reshape((len(Sensor1), 1))
Sensor2 = Sensor2.reshape((len(Sensor2), 1))
Sensor3 = Sensor3.reshape((len(Sensor3), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((Sensor1, Sensor2, Sensor3, out_seq))
# choose a number of time steps
n_steps = 4

In [99]:
print (dataset.shape)

(9, 4)


In [100]:
print (Sensor1.shape)

(9, 1)


In [101]:
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

In [102]:
print (n_features)

3


In [103]:
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dense(64, activation='relu'))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [107]:
# fit model
model.fit(X, y, epochs=1000, verbose=0)


In [108]:
# Save the model to disk
model.save(MODEL_TF)

INFO:tensorflow:Assets written to: models/model/assets


INFO:tensorflow:Assets written to: models/model/assets


In [109]:
# demonstrate prediction
#x_input = array([[200, 205, 210], [210, 215, 220], [220, 225, 230], [230, 235, 240]])
x_input = array([[150, 155, 160], [160, 165, 170], [170, 175, 180], [280, 285, 290]])
#x_input = array([[100, 105, 110], [110, 115, 120], [120, 125, 130], [130, 135, 140]])
print (x_input.shape)
x_input = x_input.reshape((1, n_steps, n_features))
print (x_input.shape)
yhat = model.predict(x_input, verbose=0)
print(yhat)

(4, 3)
(1, 4, 3)
[[175.86151]]


In [110]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()

# Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

37612